### Install dependencies

In [ ]:
# !pip install qiskit

In [ ]:
# !pip install qiskit_algorithms

In [ ]:
# !pip install qiskit_optimization

In [ ]:
# ! pip install qiskit-aer

In [ ]:
# !pip install pylatexenc

In [ ]:
# !pip install gurobipy

In [ ]:
# !pip install deap

In [ ]:
# !python -m pip install --upgrade pip

In [ ]:
# !pip install dwave-ocean-sdk

In [ ]:
# !pip install dimod

In [ ]:
# !pip install qiskit-aqua

In [ ]:
# !pip install pennylane

### Import dependencies

In [ ]:
# useful additional packages

# import pennylane as qml

from qiskit import QuantumRegister, ClassicalRegister
from qiskit.tools.monitor import job_monitor
from qiskit.circuit.library import Diagonal
from qiskit import QuantumCircuit, Aer, execute
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import Pauli
from qiskit.extensions import UnitaryGate
from qiskit import BasicAer

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import networkx as nx
import os

from qiskit.tools.visualization import plot_histogram
from qiskit.circuit.library import TwoLocal
from qiskit.circuit import Parameter
from qiskit_optimization.applications import Maxcut, Tsp
from qiskit_optimization.algorithms import MinimumEigenOptimizer


from qiskit.utils import algorithm_globals
from qiskit.primitives import Sampler

from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import Pauli, Statevector
from qiskit.result import QuasiDistribution


from scipy.optimize import minimize
from itertools import combinations
import time

In [ ]:
# UPDATED MODULES

from qiskit_algorithms.minimum_eigensolvers import QAOA, SamplingVQE, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA

from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator

from qiskit_algorithms.utils import algorithm_globals


In [ ]:
import gurobipy as gp
from gurobipy import GRB

In [ ]:
from deap import base, creator, tools, algorithms

In [ ]:
import json

In [ ]:
import random

In [ ]:
import dimod
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
from dwave.samplers import SimulatedAnnealingSampler

### Initialize the graph (problem instance)

In [ ]:
# n = 3

height,width = 3,3

image = np.array([
       [0.92,  0.87, 0.39],
       [0.93,  0.90, 0.45],
       [0.94,  0.45, 0.48]
       ])
plt.imshow(image, interpolation='nearest', cmap=plt.cm.gray)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Given data and parameters
height, width = 2, 2
image = np.array([
    [0.92, 0.35],
    [0.89, 0.45]
])

# Plot the image
plt.imshow(image, interpolation='nearest', cmap=plt.cm.gray)
plt.axis('off')

# Annotate each pixel with its position
for (i, j), val in np.ndenumerate(image):
    plt.text(j, i, f'({i},{j})', ha='center', va='center', color='grey', fontsize=20)

# Save the image with annotations
plt.savefig("2x2_v2.jpg", dpi=600, bbox_inches='tight', pad_inches=0)
plt.show()


In [ ]:
image.shape, height, width

In [ ]:
def image_to_grid_graph(gray_img, sigma=0.5):
  # Convert image to grayscale
  # gray_img = np.dot(img, [0.2989, 0.5870, 0.1140])
  h, w = gray_img.shape
  # Initialize graph nodes and edges
  nodes = np.zeros((h*w, 1))
  edges = []
  nx_elist = []
  # Compute node potentials and edge weights
  min_weight = 1
  max_weight = 0
  for i in range(h*w):
    x, y = i//w, i%w
    nodes[i] = gray_img[x,y]
    if x > 0:
      j = (x-1)*w + y
      # weight = np.exp(-dist.euclidean([gray_img[x,y]], [gray_img[x-1,y]])**2 / (2*sigma**2))
      weight = np.exp(-((gray_img[x,y] - gray_img[x-1,y])**2) / (2 * sigma**2))
      edges.append((i, j, weight))
      nx_elist.append(((x,y),(x-1,y),np.round(weight,2)))
      if min_weight>weight:min_weight=weight
      if max_weight<weight:max_weight=weight
    if y > 0:
      j = x*w + y-1
      # weight = np.exp(-dist.euclidean([gray_img[x,y]], [gray_img[x,y-1]])**2 / (2*sigma**2))
      weight = np.exp(-((gray_img[x,y] - gray_img[x,y-1])**2) / (2 * sigma**2))
      edges.append((i, j, weight))
      nx_elist.append(((x,y),(x,y-1),np.round(weight,2)))
      if min_weight>weight:min_weight=weight
      if max_weight<weight:max_weight=weight
  a=-1
  b=1
  normalized_edges = [(node1,node2,-1*np.round(((b-a)*((edge_weight-min_weight)/(max_weight-min_weight)))+a,2)) for node1,node2,edge_weight in edges]
  normalized_nx_elist = [(node1,node2,-1*np.round(((b-a)*((edge_weight-min_weight)/(max_weight-min_weight)))+a,2)) for node1,node2,edge_weight in nx_elist]
  return nodes, edges, nx_elist, normalized_edges, normalized_nx_elist



pixel_values, elist, nx_elist, normalized_elist, normalized_nx_elist = image_to_grid_graph(image)

pixel_values, elist, nx_elist, normalized_elist, normalized_nx_elist

In [ ]:
G = nx.grid_2d_graph(image.shape[0], image.shape[1])

# G.add_weighted_edges_from()
G.add_weighted_edges_from(normalized_nx_elist)
# G.add_weighted_edges_from(nx_elist)



def draw(G):
  plt.figure(figsize=(6,6))
  default_axes = plt.axes(frameon=True)
  pos = {(x,y):(y,-x) for x,y in G.nodes()}
  print("Pos:",pos)
  nx.draw_networkx(G,
                  pos=pos,
                  node_color=1-pixel_values,
                  with_labels=True,
                  node_size=3000,
                  cmap=plt.cm.Greys,
                  alpha=1,
                  ax=default_axes)
  nodes = nx.draw_networkx_nodes(G, pos, node_color=1-pixel_values,
                  node_size=3000,
                  cmap=plt.cm.Greys)
  nodes.set_edgecolor('k')
  edge_labels = nx.get_edge_attributes(G, "weight")
  print(edge_labels)
  nx.draw_networkx_edge_labels(G,
                              pos=pos,
                              edge_labels=edge_labels,
                              font_size=20)
  plt.axis('off')
  plt.savefig("2x2_graph.jpg", dpi=600, bbox_inches='tight', pad_inches=0)
draw(G)

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean

def image_to_grid_graph_full(gray_img, sigma=0.5):
    h, w = gray_img.shape
    nodes = np.zeros((h*w, 1))
    normalized_nx_elist = []

    # Compute node potentials and edge weights
    for i in range(h*w):
        x1, y1 = i // w, i % w
        nodes[i] = gray_img[x1, y1]
        for j in range(i + 1, h * w):
            x2, y2 = j // w, j % w
            pixel_diff = (gray_img[x1, y1] - gray_img[x2, y2]) ** 2
            dist = euclidean((x1, y1), (x2, y2)) ** 2
            weight = (1 - np.exp(-pixel_diff / (2 * sigma**2))) / dist
            normalized_nx_elist.append(((x1, y1), (x2, y2), np.round(weight, 2)))

    return nodes, normalized_nx_elist

# Example usage
height, width = 3, 3
image = np.array([
    [0.92, 0.87, 0.39],
    [0.93, 0.90, 0.45],
    [0.94, 0.45, 0.48]
])
pixel_values, normalized_nx_elist = image_to_grid_graph_full(image)

G = nx.Graph()
G.add_nodes_from([(i // width, i % width) for i in range(height * width)])
G.add_weighted_edges_from(normalized_nx_elist)

# Display the image
# plt.imshow(image, interpolation='nearest', cmap=plt.cm.gray)
# plt.show()
draw(G)

## Experiments

#### Generate Problem Instance

In [ ]:
def image_to_grid_graph(gray_img, sigma=0.5):
  # Convert image to grayscale
  # gray_img = np.dot(img, [0.2989, 0.5870, 0.1140])
  h, w = gray_img.shape
  # Initialize graph nodes and edges
  nodes = np.zeros((h*w, 1))
  edges = []
  nx_elist = []
  # Compute node potentials and edge weights
  min_weight = 1
  max_weight = 0
  for i in range(h*w):
    x, y = i//w, i%w
    nodes[i] = gray_img[x,y]
    if x > 0:
      j = (x-1)*w + y
      # weight = np.exp(-dist.euclidean([gray_img[x,y]], [gray_img[x-1,y]])**2 / (2*sigma**2))
      weight = 1-np.exp(-((gray_img[x,y] - gray_img[x-1,y])**2) / (2 * sigma**2))
      edges.append((i, j, weight))
      nx_elist.append(((x,y),(x-1,y),np.round(weight,2)))
      if min_weight>weight:min_weight=weight
      if max_weight<weight:max_weight=weight
    if y > 0:
      j = x*w + y-1
      # weight = np.exp(-dist.euclidean([gray_img[x,y]], [gray_img[x,y-1]])**2 / (2*sigma**2))
      weight = 1-np.exp(-((gray_img[x,y] - gray_img[x,y-1])**2) / (2 * sigma**2))
      # print('weight',weight)
      edges.append((i, j, weight))
      nx_elist.append(((x,y),(x,y-1),np.round(weight,2)))
      if min_weight>weight:min_weight=weight
      if max_weight<weight:max_weight=weight
  a=-1
  b=1
  if max_weight-min_weight:
    normalized_edges = [(node1,node2,-1*np.round(((b-a)*((edge_weight-min_weight)/(max_weight-min_weight)))+a,2)) for node1,node2,edge_weight in edges]
    normalized_nx_elist = [(node1,node2,-1*np.round(((b-a)*((edge_weight-min_weight)/(max_weight-min_weight)))+a,2)) for node1,node2,edge_weight in nx_elist]
  else:
    normalized_edges = [(node1,node2,-1*np.round(edge_weight,2)) for node1,node2,edge_weight in edges]
    normalized_nx_elist = [(node1,node2,-1*np.round(edge_weight,2)) for node1,node2,edge_weight in nx_elist]
  return nodes, edges, nx_elist, normalized_edges, normalized_nx_elist

In [ ]:
list1 = [1,1]

min_ele = min(list1)
max_ele = max(list1)

a=-1
b=1

if max_ele-min_ele:
  print([np.round(((b-a)*((ele-min_ele)/(max_ele-min_ele)))+a,2) for ele in list1])
else:
  print([np.round(ele,2) for ele in list1])

In [ ]:
def generate_problem_instance(height,width):
  image = np.random.rand(height,width)
  # plt.imshow(image, interpolation='nearest', cmap=plt.cm.gray)
  pixel_values, elist, nx_elist, normalized_elist, normalized_nx_elist = image_to_grid_graph(image)
  G = nx.grid_2d_graph(image.shape[0], image.shape[1])
  G.add_weighted_edges_from(normalized_nx_elist)
  return G, image

In [ ]:
def generate_binary_problem_instance(height,width):
  image = np.random.rand(height,width)
  # print(image)
  image[image < 0.5] = 0
  image[image >= 0.5] = 1
  # print(image)
  # plt.imshow(image, interpolation='nearest', cmap=plt.cm.gray)
  pixel_values, elist, nx_elist, normalized_elist, normalized_nx_elist = image_to_grid_graph(image)
  # print('normalized_elist',normalized_elist)
  # print('normalized_nx_elist',normalized_nx_elist
  G = nx.grid_2d_graph(image.shape[0], image.shape[1])
  G.add_weighted_edges_from(normalized_nx_elist)
  return G, image

In [ ]:
G, image = generate_binary_problem_instance(2,2)
print("G.number_of_nodes():",G.number_of_nodes())
print("nx.adjacency_matrix(G).todense() \n",nx.adjacency_matrix(G).todense())
plt.imshow(image, interpolation='nearest', cmap=plt.cm.gray)

In [ ]:
nx.adjacency_matrix(G).todense()

In [ ]:
def draw(G, image):
  pixel_values = image
  plt.figure(figsize=(8,8))
  default_axes = plt.axes(frameon=True)
  pos = {(x,y):(y,-x) for x,y in G.nodes()}
  nx.draw_networkx(G,
                  pos=pos,
                  node_color=1-pixel_values,
                  with_labels=True,
                  node_size=3000,
                  cmap=plt.cm.Greys,
                  alpha=0.8,
                  ax=default_axes)
  nodes = nx.draw_networkx_nodes(G, pos, node_color=1-pixel_values,
                  node_size=3000,
                  cmap=plt.cm.Greys)
  nodes.set_edgecolor('k')
  edge_labels = nx.get_edge_attributes(G, "weight")
  nx.draw_networkx_edge_labels(G,
                              pos=pos,
                             edge_labels=edge_labels)

In [ ]:
draw(G, image)

#### Brute Force Solver

In [ ]:
def bf_solver(G):
  n = G.number_of_nodes()
  w = nx.adjacency_matrix(G).todense()
  # print('weights matrix =',w)
  best_cost_brute = 100000
  for b in range(2**(n-1)):
    x = [int(t) for t in reversed(list(bin(b)[2:].zfill(n)))]
    cost = 0
    for i in range(n):
      for j in range(n):
        cost = cost + w[i,j]*x[i]*(1-x[j])
    if cost < best_cost_brute:
      best_cost_brute = cost
      xbest_brute = x
  return xbest_brute, best_cost_brute

#### Minimum Eigen Solver (Classical)

In [ ]:
def me2_solver(G):
  n = G.number_of_nodes()
  w = -1 * nx.adjacency_matrix(G).todense()
  max_cut = Maxcut(w)
  qp = max_cut.to_quadratic_program()
  qubitOp, offset = qp.to_ising()
  exact = MinimumEigenOptimizer(NumPyMinimumEigensolver())
  result = exact.solve(qp)
  # return result.x, result.fval
  ee = NumPyMinimumEigensolver()
  result = ee.compute_minimum_eigenvalue(qubitOp)
  x = max_cut.sample_most_likely(result.eigenstate)
  return x, qp.objective.evaluate(x)

#### Qiskit QAOA Solver

In [ ]:
def objective_value(x: np.ndarray, w: np.ndarray) -> float:
    """Compute the value of a cut.
    Args:
        x: Binary string as numpy array.
        w: Adjacency matrix.
    Returns:
        Value of the cut.
    """
    cost = 0
    for i in range(len(x)):
        for j in range(len(x)):
            cost = cost + w[i,j]*x[i]*(1-x[j])
    return cost

def bitfield(n: int, L: int) -> list[int]:
    result = np.binary_repr(n, L)
    return [int(digit) for digit in result]  # [2:] to chop off the "0b" part

def sample_most_likely(state_vector) -> np.ndarray:
    """Compute the most likely binary string from state vector.
    Args:
        state_vector: State vector or quasi-distribution.

    Returns:
        Binary string as an array of ints.
    """
    if isinstance(state_vector, QuasiDistribution):
        values = list(state_vector.values())
    else:
        values = state_vector
    n = int(np.log2(len(values)))
    k = np.argmax(np.abs(values))
    x = bitfield(k, n)
    x.reverse()
    return np.asarray(x)

In [ ]:
def qaoa_solver(G, reps=1):
  n = G.number_of_nodes()
  w = -1 * nx.adjacency_matrix(G).todense()
  max_cut = Maxcut(w)
  qp = max_cut.to_quadratic_program()
  # print(qp.prettyprint())
  qubitOp, offset = qp.to_ising()
  sampler = Sampler()
  algorithm_globals.random_seed = 123
  optimizer = COBYLA()
  qaoa = QAOA(sampler, optimizer, reps=reps)
  result = qaoa.compute_minimum_eigenvalue(qubitOp)
  x = sample_most_likely(result.eigenstate)
  return x, objective_value(x, w)

#### Quantum Annealer (D-Wave)

In [ ]:
def dwave_solver(linear, quadratic, offset = 0.0, runs=10000, **kwargs):
  """
  Solve Ising hamiltonian or qubo problem instance using dimod API for using dwave system.
  :params
  linear: dictionary of linear coefficient terms in the QUBO formulation of the CSG problem.
  quadratic: dictionary of quadratic coefficient terms in the QUBO formulation of the CSG problem.
  runs: Number of repeated executions
  :return
  sample_set: Samples and any other data returned by dimod samplers.
  """
  vartype = dimod.BINARY
  bqm = dimod.BinaryQuadraticModel(linear, quadratic, 0.0, vartype)
  start_time = time.time()
  dwave_sampler = DWaveSampler(token = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx", solver={'topology__type': 'pegasus'})
  connection_time = time.time() - start_time

  start_time = time.time()
  sampler = EmbeddingComposite(dwave_sampler)
  embedding_time = time.time() - start_time
  start_time = time.time()
  sample_set = sampler.sample(bqm, num_reads=runs)
  response_time = time.time() - start_time
  return sample_set, connection_time, embedding_time, response_time

In [ ]:
def annealer_solver(G, save_log=False, name_folder='distribution', n_samples= 2000, n_run=1, **kwargs):
  start_time = time.time()
  w = -1 * nx.adjacency_matrix(G).todense()
  max_cut = Maxcut(w)
  qp = max_cut.to_quadratic_program()
  linear = qp.objective.linear.coefficients.toarray(order=None, out=None)
  quadratic = qp.objective.quadratic.coefficients.toarray(order=None, out=None)
  linear = {int(idx):-round(value,2) for idx,value in enumerate(linear[0])}
  quadratic = {(int(iy),int(ix)):-quadratic[iy, ix] for iy, ix in np.ndindex(quadratic.shape) if iy<ix and abs(quadratic[iy, ix])!=0}
  problem_formulation_time = time.time() - start_time
  sample_set, connection_time, embedding_time, response_time = dwave_solver(linear, quadratic, offset = 0.0, runs=n_samples)
  info_dict = sample_set.info['timing'].copy()

  start_time = time.time()
  samples_df = sample_set.to_pandas_dataframe()
  sample_fetch_time = time.time() - start_time

  info_dict['problem_formulation_time'] = problem_formulation_time
  info_dict['connection_time'] = connection_time
  info_dict['embedding_time'] = embedding_time
  info_dict['response_time'] = response_time
  info_dict['sample_fetch_time'] = sample_fetch_time
  return samples_df, info_dict

#### New NISQ Solver

In [ ]:
def R(theta):
  if abs(theta) > 2*np.pi or abs(theta) < 0:
    theta = abs(theta) - (np.floor(abs(theta)/(2*np.pi))*(2*np.pi))
  return 0 if 0 <= theta < np.pi else 1

def cost_fn(params, hermitian_observable):
  global optimization_iteration_count
  optimization_iteration_count += 1
  N = int(np.log2(len(params)))
  circuit_psi = QuantumCircuit(N)
  for i in range(N):
    circuit_psi.h(i)
  diagonal_elements = [np.exp(1j * np.pi * R(param)) for param in params]
  diag_gate = Diagonal(diagonal_elements)
  circuit_psi.append(diag_gate, range(N))
  op_observable = SparsePauliOp.from_operator(hermitian_observable)
  cost = Estimator().run(circuit_psi, op_observable).result().values[0]
  print(f'@ Iteration {optimization_iteration_count} Cost :',cost)
  return cost

def decode(optimal_params):
  return [R(param) for param in optimal_params]

In [ ]:
def new_nisq_ga_solver2(G, population_size=50, crossover_probability=0.7, mutation_probability=0.2, number_of_generations=50):
    n = G.number_of_nodes()
    w = nx.adjacency_matrix(G).todense()
    D_G = np.diag(list(dict(G.degree()).values()))
    A_G = w
    L_G = D_G - A_G
    n_padding = (2**int(np.ceil(np.log2(n))) - n)
    L_G = np.pad(L_G, [(0, n_padding), (0, n_padding)], mode='constant')
    H = L_G

    # Genetic Algorithm Setup
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    toolbox.register("attr_float", random.uniform, 0.5, 2*np.pi)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # Evaluate function with padding
    def evaluate_with_padding(individual):
        padded_individual = individual + [0] * n_padding  # Append constant values for padding
        return cost_fn(padded_individual, H),

    toolbox.register("evaluate", evaluate_with_padding)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # Run Genetic Algorithm
    population = toolbox.population(n=population_size)
    algorithms.eaSimple(population, toolbox, cxpb=crossover_probability, mutpb=mutation_probability, ngen=number_of_generations, verbose=False)

    # Extract the best solution
    best_ind = tools.selBest(population, 1)[0]
    optimal_params = best_ind + [0] * n_padding  # Append constant values for padding
    expectation_value = best_ind.fitness.values[0]

    # Decode and calculate cut value
    x = np.real(decode(optimal_params))
    x = x[:n]
    cut_value = objective_value(x, w)

    return x, expectation_value, cut_value


In [ ]:
def new_nisq_algo_solver(G, optimizer_method = 'Genetic', initial_params_seed=123):
  global optimization_iteration_count
  optimization_iteration_count = 0
  if optimizer_method == 'Genetic':
    x, expectation_value, cut_value = new_nisq_ga_solver2(G,
                                                         population_size = 50,
                                                         crossover_probability = 0.7,
                                                         mutation_probability = 0.2,
                                                         number_of_generations = 50)
    success_flag = True
  else:
    n = G.number_of_nodes()
    w = nx.adjacency_matrix(G).todense()
    D_G = np.diag(list(dict(G.degree()).values()))
    A_G = w
    L_G = D_G - A_G
    n_padding = (2**int(np.ceil(np.log2(n)))-n)
    L_G = np.pad(L_G, [(0, n_padding), (0, n_padding) ], mode='constant')
    H = L_G
    # if initial_params is None:
    np.random.seed(seed=initial_params_seed)
    initial_params = np.random.uniform(low=0.5, high=2*np.pi , size=(n+n_padding))
    # optimizers_scipy = ['Nelder-Mead', 'Powell', 'CG', 'BFGS', 'L-BFGS-B', 'TNC', 'COBYLA', 'SLSQP']
    # print(optimizer_method)
    options = {}
    result = minimize(
        fun=cost_fn,
        x0=initial_params,  # Initial guess for the parameters
        args=H,    # Additional arguments passed to evaluate_cost
        method=optimizer_method,
        options=options)
    
    optimal_params, expectation_value = result.x, result.fun
    x = np.real(decode(optimal_params))
    x = x[:n]
    cut_value = objective_value(x, w)
    success_flag = result.success
  return success_flag, x, expectation_value, cut_value

#### Experiments for New NISQ

In [ ]:
# max_layers = 20
# n_shots = 65536

# optimization_max_iter = 10000


# heights = np.arange(2, 17).tolist()
heights = [2,4,8,16]
heights = [16]

# scipy_optimizer_methods = ["COBYLA", "Nelder-Mead", "Powell", "CG", "BFGS", "Newton-CG", "L-BFGS-B", "SLSQP", "trust-const", "dogleg", "trust-ncg", "trust-exact", "trust-krylov"]
# scipy_optimizer_methods = ["COBYLA", "Nelder-Mead", "Powell", "CG", "BFGS", "L-BFGS-B", "SLSQP"]
# scipy_optimizer_methods = ["COBYLA", "Powell", "CG", "BFGS", "L-BFGS-B", "SLSQP"]
scipy_optimizer_methods = ["COBYLA"]
# optimizer_method = 'Powell'

seed = 111
initial_params_seed = 123

ultimate_valid_probabilities = []
penultimate_valid_probabilities = []
optimization_iteration_count = 0

base_path = './'
report_filename = base_path + 'NewNISQv2_' +  str(seed) + '_.txt'
# report_filename = base_path + 'SampleTestMinimalEncoding_' +  str(seed) + '_' + str(n_shots) + '.txt'


In [ ]:
report_filename

In [ ]:

for height in heights:
  width = height
  print(f'height: {height}, width: {width}, n: {height*width}')
  # G, image = generate_binary_problem_instance(height, width)
  np.random.seed(seed=seed)
  G, image = generate_problem_instance(height, width)
  G, image = generate_binary_problem_instance(height, width)
  print("Image Generated: ",image)
  # plt.imshow(image, cmap=plt.cm.gray)
  # plt.show()

  for optimizer_method in scipy_optimizer_methods:
    # if G.number_of_nodes() <= max_layers:
      # max_layers = G.number_of_nodes()
    # else:
      # max_layers = 30
    # print("Maximum number of layers :", max_layers)
    # for n_layers in range(1, 1+1,1):
    #   nc = len(G.nodes())
    #   nr = ceil(log2(nc))
    #   nq = nr + 1
      # np.random.seed(seed=initial_params_seed)
      # initial_params = np.random.uniform(low=-np.pi, high=np.pi, size=nq*n_layers)
    print(f"Executing QC with {optimizer_method} optimizer for {height}*{height} image.")
    # try:
    start_time = time.time()
      # success_flag, minimal_encoding_solution, minimal_encoding_value, minimal_encoding_cut_value, optimal_params = minimal_encoding(G, 
                                                                                                                    # initial_params, 
                                                                                                                    # n_layers = n_layers,
                                                                                                                    # max_iter = optimization_max_iter,
                                                                                                                    # optimizer_method = scipy_optimizer_method)
    success_flag, new_nisq_solution, new_nisq_expectation_value, new_nisq_cut_value = new_nisq_algo_solver(G, optimizer_method = optimizer_method, initial_params_seed=initial_params_seed)
    new_nisq_tte = (time.time() - start_time)
    print(new_nisq_solution, new_nisq_expectation_value, new_nisq_cut_value)
    # except:
      # print(f"Execution Failed for {n_layers} layers and {scipy_optimizer_method} optimizer for {height}*{height} image.")
      # continue
    # print("New NISQ done for",scipy_optimizer_method,"optimizer with a success status :", success_flag)
    # print(f"Appending the results of {height}*{height} image using QC with {n_layers} layers and {scipy_optimizer_method} optimizer.")
    row = []
    row.append(int(G.number_of_nodes()))
    row.append(int(height))
    row.append(int(width))
    row.append(success_flag)
    row.append(''.join(map(str, map(int, (new_nisq_solution)))))
    row.append(np.round(new_nisq_tte,6))
    row.append(np.round(new_nisq_cut_value,4))
    row.append(np.round(new_nisq_expectation_value,4))
    row.append(optimization_iteration_count)
    row.append(optimizer_method)
    # row.append(''.join(map(str, map(float, (optimal_params)))))
    report_file_obj = open(os.path.join(report_filename),'a+')
    report_file_obj.write('__'.join(map(str,row))+'\n')
    report_file_obj.close()
    # plt.imshow(decode_binary_string(minimal_encoding_solution, height,height), cmap=plt.cm.gray)
    # plt.show()
    # if success_flag:
      # break
  print('\n')

In [ ]:
def truncate_long_string(s, max_length=20):
    return s if len(s) <= max_length else s[:max_length] + "..."

def style_df(df):
    # Styles for hover and headers
    cell_hover = {'selector': 'tr:hover', 'props': [('background-color', '#707070')]}  # Dark pale color for hover
    headers = {'selector': 'th', 'props': 'background-color: #1D1D1D; color: white;'}

    # Column separator style
    column_separator = {'selector': 'td, th', 'props': 'border-right: 1px solid white;'}

    # Apply the styles
    styled_df = df.style.set_table_styles([cell_hover, headers, column_separator])

    # Style to hide the border for the last column to prevent a double line at the end of the table
    hide_last_border = {'selector': 'td:last-child, th:last-child', 'props': 'border-right: none;'}

    # Combine all styles
    styles = [cell_hover, headers, column_separator, hide_last_border]
    styled_df = styled_df.set_table_styles(styles, overwrite=False)

    return styled_df

In [ ]:
base_path = './'
report_filename = base_path + 'minimalEncoding_' +  str(seed) + '_' + str(16384) + '.txt'
# report_filename = base_path + 'minimalEncoding_' +  str(seed) + '_' + str(32768) + '.txt'
# report_filename = base_path + 'SampleTestMinimalEncoding_' +  str(seed) + '_' + str(8192) + '.txt'
report_file_obj = open(report_filename,'r')
table_contents = [line.replace('\n','').split('__') for line in report_file_obj.readlines()]

base_cols = ['No. of Pixels', 'Height', 'Width']
sub_cols = ['','','']

extra_sub_cols = ['Optimization Success','Result', 'TTE', 'Layers', 'Min-Cut Cost', 'Optimization Cost', 'Iterations', 'Optimizer Method']
base_cols = base_cols+['Minimal Encoding Solver']*len(extra_sub_cols)
sub_cols=sub_cols+extra_sub_cols

column_arrays = [base_cols, sub_cols]

#df = pd.DataFrame(table_contents, columns=table_headers)
df = pd.DataFrame(table_contents, columns=pd.MultiIndex.from_arrays(column_arrays))

for col in df.columns:
  if len(col) == 2 and col[1] == 'Result':
    df[col] = df[col].apply(lambda x: truncate_long_string(x))

styled_df = style_df(df)#.apply(highlight_row, axis=1)




styled_df

In [ ]:
# G, image = generate_binary_problem_instance(2,2)
# plt.imshow(image, cmap=plt.cm.gray)
# draw(G, image)

In [ ]:
# # scipy_optimizer_method = "Powell"

# height = 4

# seed = 111
# np.random.seed(seed=seed)
# G, image = generate_binary_problem_instance(height, height)

# start = time.time()
# new_nisq_solution, new_nisq_expectation_value, new_nisq_cut_value = new_nisq_ga_solver(G, population_size = min(10,int(2*G.number_of_nodes())),
#                                                                                        crossover_probability = 0.5,
#                                                                                        mutation_probability = 0.5,
#                                                                                        number_of_generations = min(10,int(2*G.number_of_nodes())))
# new_nisq_solution, new_nisq_expectation_value, new_nisq_cut_value, time.time()-start

In [ ]:
# start = time.time()
# new_nisq_solution, new_nisq_expectation_value, new_nisq_cut_value = new_nisq_ga_solver2(G, population_size = min(10,int(2*G.number_of_nodes())),
#                                                                                         crossover_probability = 0.5,
#                                                                                         mutation_probability = 0.5,
#                                                                                         number_of_generations = min(10,int(2*G.number_of_nodes())))
# new_nisq_solution, new_nisq_expectation_value, new_nisq_cut_value, time.time()-start

#### Minimal Encoding

In [ ]:
from qiskit import Aer, execute
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector
import numpy as np

In [ ]:
from qiskit.circuit import ParameterVector

In [ ]:
from qiskit import QuantumCircuit
from math import log2, ceil

In [ ]:
def get_circuit(nq, n_layers):
  theta = ParameterVector('θ', length=nq*n_layers)
  qc = QuantumCircuit(nq)
  qc.h(range(nq))

  for layer_i in range(n_layers):
    if layer_i%2:
      for qubit_i in range(nq - 1):
        qc.cx(qubit_i, (qubit_i + 1)%nq)
    for qubit_i in range(nq):
      qc.ry(theta[(nq*layer_i)+qubit_i], qubit_i)
  return qc

In [ ]:
# def measure_probabilities(qc, nq, backend=Aer.get_backend('qasm_simulator')):
#   qc.measure_all()
#   job = execute(qc, backend, shots=1024)
#   result = job.result()
#   counts = result.get_counts(qc)
#   probabilities = {state: count / 1024 for state, count in counts.items()}
#   return probabilities

def get_projectors(probabilities, n_c):
  P = [0] * n_c
  P1 = [0] * n_c
  for k, v in probabilities.items():
    index = int(k[1:], 2)
    P[index] += v
    if k[0] == '1':
      P1[index] += v
  return P, P1

def evaluate_cost(params, circuit, qubo_matrix):
  global ultimate_valid_probabilities
  global penultimate_valid_probabilities
  global optimization_iteration_count
  global n_shots
  optimization_iteration_count += 1
  # Create a copy of the circuit to avoid modifying the original
  working_circuit = circuit.copy()

  # Apply measurements
  working_circuit.measure_all()

  # Bind the parameters to the circuit
  bound_circuit = working_circuit.assign_parameters(params)

  # Run the circuit on a simulator

  # n_shots = 8192
  # n_shots = 16384
  # n_shots = 32768
  simulator = Aer.get_backend('qasm_simulator')
  job = execute(bound_circuit, simulator, shots=n_shots)
  result = job.result()
  counts = result.get_counts()

  # Compute the probabilities of the states
  probabilities = {state: counts[state] / n_shots for state in counts}
  # print('probabilities',probabilities)
  # print('ultimate_valid_probabilities',ultimate_valid_probabilities)
  # print('penultimate_valid_probabilities',penultimate_valid_probabilities)
  n_c = 2**(len(list(probabilities.keys())[0])-1)
  # n_c = qubo.shape[0]
  P, P1 = get_projectors(probabilities, n_c)

  if 0 in P:
    return 100000
  else:
    # Update the last valid probabilities
    penultimate_valid_probabilities = ultimate_valid_probabilities.copy()
    ultimate_valid_probabilities = probabilities.copy()

  cost = 0
  for i in range(len(qubo_matrix)):
    for j in range(len(qubo_matrix)):
      if i==j:
        cost += qubo_matrix[i][j] * P1[i]/P[i]
      else:
        cost += (qubo_matrix[i][j]*P1[i]*P1[j])/(P[i]*P[j])
  # binary_string = decode_probabilities(probabilities)
  # print('Binary String :', binary_string)
  # min_cut_cost = 0
  # for i in range(len(binary_string)):
    # for j in range(len(binary_string)):
      # min_cut_cost = min_cut_cost + qubo_matrix[i][j]*binary_string[i]*(1-binary_string[j])
  # print('min_cut_cost :', min_cut_cost)
  print(f'@ Iteration {optimization_iteration_count} Cost :',cost)
  return cost

def get_final_measurement_binary_string(circuit, params):
  # Create a copy of the circuit to avoid modifying the original
  working_circuit = circuit.copy()
  # Apply measurements
  working_circuit.measure_all()
  # Bind the parameters to the circuit
  bound_circuit = working_circuit.assign_parameters(params)
  # Run the circuit on a simulator
  simulator = Aer.get_backend('qasm_simulator')
  job = execute(bound_circuit, simulator, shots=1024)
  result = job.result()
  counts = result.get_counts()
  # Compute the probabilities of the states
  probabilities = {state: counts[state] / 1024 for state in counts}
  return probabilities

def decode_probabilities(probabilities):
  binary_solution = []
  n_r = len(list(probabilities.keys())[0])-1
  n_c = 2**(n_r)
  for i in range(n_c):
    if '0' + format(i, 'b').zfill(n_r) in probabilities and '1' + format(i, 'b').zfill(n_r) in probabilities:
      if probabilities['0' + format(i, 'b').zfill(n_r)] > probabilities['1' + format(i, 'b').zfill(n_r)]:
        binary_solution.append(0)
      else:
        binary_solution.append(1)
    elif '0' + format(i, 'b').zfill(n_r) in probabilities:
      binary_solution.append(0)
    elif '1' + format(i, 'b').zfill(n_r) in probabilities:
      binary_solution.append(1)
  return binary_solution

def decode_binary_string(x, height, width):
  mask = np.zeros([height, width])
  for index,segment in enumerate(x):
    mask[index//width,index%width]=segment
  return mask

In [ ]:


# For a larger number of shots, rather than measuring the probabilities, use statevector simulator, we get the statevectors itself instead of proababilities, squaring the amplitudes we get probabilities.



In [ ]:
# x_1 = 'state of the ancilla qubit whose register qubits are in the state (00) with high probability' = 1
# x_2 = 'state of the ancilla qubit whose register qubits are in the state (01) with high probability' = 1
# x_3 = 'state of the ancilla qubit whose register qubits are in the state (10) with high probability' = 0
# x_4 = 'state of the ancilla qubit whose register qubits are in the state (11) with high probability' = 0

In [ ]:
def minimal_encoding(G, initial_params, n_layers = 1, max_iter = 2000, optimizer_method = 'COBYLA'):

  w = nx.adjacency_matrix(G).todense()
  max_cut = Maxcut(-w)
  qp = max_cut.to_quadratic_program()
  linear = qp.objective.linear.coefficients.toarray(order=None, out=None)
  quadratic = qp.objective.quadratic.coefficients.toarray(order=None, out=None)
  linear = {int(idx):-round(value,2) for idx,value in enumerate(linear[0])}
  quadratic = {(int(iy),int(ix)):-quadratic[iy, ix] for iy, ix in np.ndindex(quadratic.shape) if iy<ix and abs(quadratic[iy, ix])!=0}
  nc = len(linear)
  nr = ceil(log2(nc))
  nq = nr + 1
  qubo = np.zeros((nc, nc))
  np.fill_diagonal(qubo, list(linear.values()))
  for (i, j), value in quadratic.items():
    qubo[i, j] = qubo[j, i] = value
  qc = get_circuit(nq, n_layers)
  # initial_params = np.random.uniform(low=-np.pi, high=np.pi, size=nq*n_layers)
  global ultimate_valid_probabilities
  global penultimate_valid_probabilities
  global optimization_iteration_count
  ultimate_valid_probabilities = []
  penultimate_valid_probabilities = []
  optimization_iteration_count = 0
  
  # Correct way to call the minimize function
  options = {'maxiter': max_iter, 'rhoend':1e-8, 'maxfun':max_iter, 'disp':None, 'catol':2e-8}
  optimization_result = minimize(
      fun=evaluate_cost,
      x0=initial_params,  # Initial guess for the parameters
      args=(qc, qubo),    # Additional arguments passed to evaluate_cost
      method=optimizer_method,
      options=options)
  if not len(ultimate_valid_probabilities):
    return False, [0]*nc, 0, 0, [0]*initial_params
  binary_string_solution = decode_probabilities(ultimate_valid_probabilities)
  minimization_cost = optimization_result.fun
  optimal_params = optimization_result.x
  cut_cost = 0
  cut_cost = objective_value(np.array(list(map(int,binary_string_solution))), w)
  # print("W.shape :",w.shape)
  # print("len(binary_string_solution) :",len(binary_string_solution))
  # for i in range(len(binary_string_solution)):
    # for j in range(len(binary_string_solution)):
      # cut_cost = cut_cost + w[i,j]*binary_string_solution[i]*(1-binary_string_solution[j])
  return optimization_result.success, binary_string_solution, minimization_cost, cut_cost, optimal_params


In [ ]:
def truncate_long_string(s, max_length=20):
    return s if len(s) <= max_length else s[:max_length] + "..."


def style_df(df):
    # Styles for hover and headers
    cell_hover = {'selector': 'tr:hover', 'props': [('background-color', '#707070')]}  # Dark pale color for hover
    headers = {'selector': 'th', 'props': 'background-color: #1D1D1D; color: white;'}

    # Column separator style
    column_separator = {'selector': 'td, th', 'props': 'border-right: 1px solid white;'}

    # Apply the styles
    styled_df = df.style.set_table_styles([cell_hover, headers, column_separator])

    # Style to hide the border for the last column to prevent a double line at the end of the table
    hide_last_border = {'selector': 'td:last-child, th:last-child', 'props': 'border-right: none;'}

    # Combine all styles
    styles = [cell_hover, headers, column_separator, hide_last_border]
    styled_df = styled_df.set_table_styles(styles, overwrite=False)

    return styled_df

#### Experiments for Minimal Encoding

In [ ]:
max_layers = 20
n_shots = 65536

optimization_max_iter = 10000


# heights = np.arange(2, 17).tolist()
heights = [2,4,8,16]
heights = [2,4,8]
# heights = [8]

# scipy_optimizer_methods = ["COBYLA", "Nelder-Mead", "Powell", "CG", "BFGS", "Newton-CG", "L-BFGS-B", "SLSQP", "trust-const", "dogleg", "trust-ncg", "trust-exact", "trust-krylov"]
# scipy_optimizer_methods = ["COBYLA", "Nelder-Mead", "Powell", "CG", "BFGS", "L-BFGS-B", "SLSQP"]
# scipy_optimizer_methods = ["COBYLA", "Powell", "CG", "BFGS", "L-BFGS-B", "SLSQP"]
scipy_optimizer_methods = ["COBYLA"]
# scipy_optimizer_methods = ["COBYLA", "CG", "BFGS", "L-BFGS-B", "SLSQP"]
# scipy_optimizer_methods = ["Genetic"]


seed = 222
initial_params_seed = 123

ultimate_valid_probabilities = []
penultimate_valid_probabilities = []
optimization_iteration_count = 0

base_path = './'
report_filename = base_path + 'minimalEncoding_' +  str(seed) + '_' + str(n_shots) + '.txt'
# report_filename = base_path + 'SampleTestMinimalEncoding_' +  str(seed) + '_' + str(n_shots) + '.txt'


In [ ]:
report_filename

In [ ]:
# height = 2
# width = height
# np.random.seed(seed=seed)
# G, image = generate_problem_instance(height, width)
# w = nx.adjacency_matrix(G).todense()
# w

In [ ]:
# gurobi_solution, gurobi_value = gurobi_mincut_solver(G)

# print(''.join(map(str, map(int, (gurobi_solution)))))
# print(gurobi_value)


In [ ]:
seeds = [222,333,444,555]
seeds = [222]
seeds

In [ ]:
for seed in seeds:
  base_path = './'
  report_filename = base_path + 'minimalEncodingv3_' +  str(seed) + '_' + str(n_shots) + '.txt'
  for height in heights:
    width = height
    print(f'height: {height}, width: {width}, n: {height*width}')
    # G, image = generate_binary_problem_instance(height, width)
    np.random.seed(seed=seed)
    G, image = generate_problem_instance(height, width)
    print("Image Generated: ",image)
    # plt.imshow(image, cmap=plt.cm.gray)
    # plt.show()

    for scipy_optimizer_method in scipy_optimizer_methods:
      # if G.number_of_nodes() <= max_layers:
        # max_layers = G.number_of_nodes()
      # else:
        # max_layers = 30
      print("Maximum number of layers :", max_layers)
      for n_layers in range(6, max_layers+1,1):
        nc = len(G.nodes())
        nr = ceil(log2(nc))
        nq = nr + 1
        np.random.seed(seed=initial_params_seed)
        initial_params = np.random.uniform(low=-np.pi, high=np.pi, size=nq*n_layers)
        print(f"Executing QC with {n_layers} layers and {scipy_optimizer_method} optimizer for {height}*{height} image.")
        try:
          start_time = time.time()
          success_flag, minimal_encoding_solution, minimal_encoding_value, minimal_encoding_cut_value, optimal_params = minimal_encoding(G,
                                                                                                                        initial_params,
                                                                                                                        n_layers = n_layers,
                                                                                                                        max_iter = optimization_max_iter,
                                                                                                                        optimizer_method = scipy_optimizer_method)
          minimal_encoding_tte = (time.time() - start_time)
        except:
          print(f"Execution Failed for {n_layers} layers and {scipy_optimizer_method} optimizer for {height}*{height} image.")
          continue
        print("New NISQ done for",scipy_optimizer_method,"optimizer with a success status :", success_flag)
        print(f"Appending the results of {height}*{height} image using QC with {n_layers} layers and {scipy_optimizer_method} optimizer.")
        row = []
        row.append(int(G.number_of_nodes()))
        row.append(int(height))
        row.append(int(width))
        row.append(success_flag)
        row.append(''.join(map(str, map(int, (minimal_encoding_solution)))))
        row.append(np.round(minimal_encoding_tte,6))
        row.append(n_layers)
        row.append(np.round(minimal_encoding_cut_value,4))
        row.append(np.round(minimal_encoding_value,4))
        row.append(optimization_iteration_count)
        row.append(scipy_optimizer_method)
        # row.append(''.join(map(str, map(float, (optimal_params)))))
        report_file_obj = open(os.path.join(report_filename),'a+')
        report_file_obj.write('__'.join(map(str,row))+'\n')
        report_file_obj.close()
        # plt.imshow(decode_binary_string(minimal_encoding_solution, height,height), cmap=plt.cm.gray)
        # plt.show()
        # if success_flag:
          # break
    print('\n')

In [ ]:
def truncate_long_string(s, max_length=20):
    return s if len(s) <= max_length else s[:max_length] + "..."

def style_df(df):
    # Styles for hover and headers
    cell_hover = {'selector': 'tr:hover', 'props': [('background-color', '#707070')]}  # Dark pale color for hover
    headers = {'selector': 'th', 'props': 'background-color: #1D1D1D; color: white;'}

    # Column separator style
    column_separator = {'selector': 'td, th', 'props': 'border-right: 1px solid white;'}

    # Apply the styles
    styled_df = df.style.set_table_styles([cell_hover, headers, column_separator])

    # Style to hide the border for the last column to prevent a double line at the end of the table
    hide_last_border = {'selector': 'td:last-child, th:last-child', 'props': 'border-right: none;'}

    # Combine all styles
    styles = [cell_hover, headers, column_separator, hide_last_border]
    styled_df = styled_df.set_table_styles(styles, overwrite=False)

    return styled_df

In [ ]:
seed = 111

In [ ]:
base_path = './'
report_filename = base_path + 'minimalEncoding_' +  str(seed) + '_' + str(65536) + '.txt'
# report_filename = base_path + 'minimalEncoding_' +  str(seed) + '_' + str(32768) + '.txt'
# report_filename = base_path + 'SampleTestMinimalEncoding_' +  str(seed) + '_' + str(8192) + '.txt'
report_file_obj = open(report_filename,'r')
table_contents = [line.replace('\n','').split('__') for line in report_file_obj.readlines()]

base_cols = ['No. of Pixels', 'Height', 'Width']
sub_cols = ['','','']

extra_sub_cols = ['Optimization Success','Result', 'TTE', 'Layers', 'Min-Cut Cost', 'Optimization Cost', 'Iterations', 'Optimizer Method']
base_cols = base_cols+['Minimal Encoding Solver']*len(extra_sub_cols)
sub_cols=sub_cols+extra_sub_cols

column_arrays = [base_cols, sub_cols]

#df = pd.DataFrame(table_contents, columns=table_headers)
df = pd.DataFrame(table_contents, columns=pd.MultiIndex.from_arrays(column_arrays))

for col in df.columns:
  if len(col) == 2 and col[1] == 'Result':
    df[col] = df[col].apply(lambda x: truncate_long_string(x))

styled_df = style_df(df)#.apply(highlight_row, axis=1)




styled_df

In [ ]:
# Convert 'Minimal Encoding Solver' and 'Optimization Cost' columns to float
df[['Minimal Encoding Solver', 'Optimization Cost']] = df[['Minimal Encoding Solver', 'Optimization Cost']].astype(float)

In [ ]:
import cv2
from PIL import Image

In [ ]:
def forest_segmentation(image_path, downsampling_factor = 64, edge_weight_metric = image_to_grid_graph, solver = minimal_encoding):
  d = downsampling_factor

  img = cv2.imread(image_path)
  img = cv2.medianBlur(img, 31)
  lower_green = np.array([35, 50, 50])
  upper_green = np.array([90, 255, 255])
  darkgreen_mask = cv2.inRange(cv2.cvtColor(img, cv2.COLOR_BGR2HSV), lower_green, upper_green)
  darkgreen_mask = darkgreen_mask/255
  darkgreen_mask[darkgreen_mask == 0] = -1


  img = np.mean(img, axis = 2)/255
  img = (1-img)*darkgreen_mask

  height, width = img.shape
  new_width = width // d
  new_height = height // d
  img = cv2.resize(img, [new_height,new_width])

  pixel_values, elist, nx_elist, normalized_elist, normalized_nx_elist = edge_weight_metric(img)

  G = nx.grid_2d_graph(img.shape[0], img.shape[1])
  G.add_weighted_edges_from(normalized_nx_elist)

  nc = len(G.nodes())
  nr = ceil(log2(nc))
  nq = nr + 1
  np.random.seed(seed=initial_params_seed)
  initial_params = np.random.uniform(low=-np.pi, high=np.pi, size=nq*n_layers)
  print(f"Executing QC with {n_layers} layers and {scipy_optimizer_method} optimizer for {height}*{height} image.")
  try:
    start_time = time.time()
    success_flag, minimal_encoding_solution, minimal_encoding_value, minimal_encoding_cut_value, optimal_params = minimal_encoding(G,
                                                                                                                  initial_params,
                                                                                                                  n_layers = 5,
                                                                                                                  max_iter = optimization_max_iter,
                                                                                                                  optimizer_method = 'COBYLA')
    minimal_encoding_tte = (time.time() - start_time)
  except:
    print(f"Execution Failed for {n_layers} layers and {scipy_optimizer_method} optimizer for {height}*{height} image.")

  output_mask = decode_binary_string(minimal_encoding_solution, img.shape[0], img.shape[1])
  # print(output_mask)

  mask = output_mask.repeat(d, axis=0).repeat(d, axis=1)
  return mask, success_flag, minimal_encoding_solution, minimal_encoding_value, minimal_encoding_cut_value, optimal_params, minimal_encoding_tte

In [ ]:
imgs_paths = ["/content/Forest Segmented/Forest Segmented/images/111335_sat_00.jpg",
            "/content/Forest Segmented/Forest Segmented/images/111335_sat_02.jpg",
            "/content/Forest Segmented/Forest Segmented/images/10452_sat_18.jpg",
            "/content/Forest Segmented/Forest Segmented/images/10452_sat_08.jpg"]


imgs_paths = ["forest/10452_sat_08.jpg"]

for image_path in imgs_paths:
  image = Image.open(image_path)
  plt.imshow(np.asarray(image))
  plt.axis('off')
  plt.show()

  mask, success_flag, minimal_encoding_solution, minimal_encoding_value, minimal_encoding_cut_value, optimal_params, minimal_encoding_tte = forest_segmentation(image_path)
  plt.imshow(np.asarray(mask))
  plt.axis('off')
  plt.show()

plt.figure(figsize=(10, 5))  # Adjust the figure size as needed

# First subplot for the first image
plt.subplot(1, 2, 1)  # 1 row, 2 columns, first subplot
plt.imshow(np.asarray(image))  # Adjust colormap as needed
plt.axis('off')  # Hide the axis

# Second subplot for the second image
plt.subplot(1, 2, 2)  # 1 row, 2 columns, second subplot
plt.imshow(np.asarray(mask), cmap='gray')  # Adjust colormap as needed
plt.axis('off')  # Hide the axis

# Display the figure with the two images
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))  # Adjust the figure size as needed

# First subplot for the first image
plt.subplot(1, 2, 1)  # 1 row, 2 columns, first subplot
plt.imshow(np.asarray(image))  # Adjust colormap as needed
plt.axis('off')  # Hide the axis

# Second subplot for the second image
plt.subplot(1, 2, 2)  # 1 row, 2 columns, second subplot
plt.imshow(np.asarray(mask), cmap='gray')  # Adjust colormap as needed
plt.axis('off')  # Hide the axis

# Display the figure with the two images
plt.show()

In [ ]:
imgs_paths = ["forest/111335_sat_00.jpg"]

for image_path in imgs_paths:
  image = Image.open(image_path)
  plt.imshow(np.asarray(image))
  plt.axis('off')
  plt.show()

  mask, success_flag, minimal_encoding_solution, minimal_encoding_value, minimal_encoding_cut_value, optimal_params, minimal_encoding_tte = forest_segmentation(image_path)
  plt.imshow(np.asarray(mask))
  plt.axis('off')
  plt.show()


plt.figure(figsize=(10, 5))  # Adjust the figure size as needed

# First subplot for the first image
plt.subplot(1, 2, 1)  # 1 row, 2 columns, first subplot
plt.imshow(np.asarray(image))  # Adjust colormap as needed
plt.axis('off')  # Hide the axis

# Second subplot for the second image
plt.subplot(1, 2, 2)  # 1 row, 2 columns, second subplot
plt.imshow(np.asarray(mask), cmap='gray')  # Adjust colormap as needed
plt.axis('off')  # Hide the axis

# Display the figure with the two images
plt.show()

In [ ]:
imgs_paths = ["forest/111335_sat_02.jpg"]

for image_path in imgs_paths:
  image = Image.open(image_path)
  plt.imshow(np.asarray(image))
  plt.axis('off')
  plt.show()

  mask, success_flag, minimal_encoding_solution, minimal_encoding_value, minimal_encoding_cut_value, optimal_params, minimal_encoding_tte = forest_segmentation(image_path)
  plt.imshow(np.asarray(mask))
  plt.axis('off')
  plt.show()


plt.figure(figsize=(10, 5))  # Adjust the figure size as needed

# First subplot for the first image
plt.subplot(1, 2, 1)  # 1 row, 2 columns, first subplot
plt.imshow(np.asarray(image))  # Adjust colormap as needed
plt.axis('off')  # Hide the axis

# Second subplot for the second image
plt.subplot(1, 2, 2)  # 1 row, 2 columns, second subplot
plt.imshow(np.asarray(mask), cmap='gray')  # Adjust colormap as needed
plt.axis('off')  # Hide the axis

# Display the figure with the two images
plt.show()

In [ ]:
imgs_paths = ["forest/10452_sat_18.jpg"]

for image_path in imgs_paths:
  image = Image.open(image_path)
  plt.imshow(np.asarray(image))
  plt.axis('off')
  plt.show()

  mask, success_flag, minimal_encoding_solution, minimal_encoding_value, minimal_encoding_cut_value, optimal_params, minimal_encoding_tte = forest_segmentation(image_path)
  plt.imshow(np.asarray(mask))
  plt.axis('off')
  plt.show()


plt.figure(figsize=(10, 5))  # Adjust the figure size as needed

# First subplot for the first image
plt.subplot(1, 2, 1)  # 1 row, 2 columns, first subplot
plt.imshow(np.asarray(image))  # Adjust colormap as needed
plt.axis('off')  # Hide the axis

# Second subplot for the second image
plt.subplot(1, 2, 2)  # 1 row, 2 columns, second subplot
plt.imshow(np.asarray(mask), cmap='gray')  # Adjust colormap as needed
plt.axis('off')  # Hide the axis

# Display the figure with the two images
plt.show()

#### Gurobi Solver

In [ ]:
def gurobi_mincut_solver(G):
  w = -1 * nx.adjacency_matrix(G).todense()
  max_cut = Maxcut(w)
  qp = max_cut.to_quadratic_program()
  linear = qp.objective.linear.coefficients.toarray(order=None, out=None)
  quadratic = qp.objective.quadratic.coefficients.toarray(order=None, out=None)

  linear = {int(idx):-round(value,2) for idx,value in enumerate(linear[0])}
  quadratic = {(int(iy),int(ix)):-quadratic[iy, ix] for iy, ix in np.ndindex(quadratic.shape) if iy<ix and abs(quadratic[iy, ix])!=0}

  qubo_matrix = np.zeros([len(linear),len(linear)])
  for key,value in linear.items():
    qubo_matrix[int(key),int(key)] = value
  for key,value in quadratic.items():
    qubo_matrix[int(key[0]),int(key[1])] = value/2
    qubo_matrix[int(key[1]),int(key[0])] = value/2

  #size of the QUBO matrix
  n = qubo_matrix.shape[0]
  model = gp.Model()

  #initialize binary variables
  x = model.addVars(n, vtype=GRB.BINARY)

  #objective function to minimize the QUBO matrix
  obj_expr = gp.quicksum(qubo_matrix[i, j] * x[i] * x[j] for i in range(n) for j in range(n))
  model.setObjective(obj_expr)

  model.setParam('OutputFlag', 0)

  model.optimize()

  if model.status == GRB.OPTIMAL:
    solution = [int(x[i].X) for i in range(n)]
    binary_string = ''.join(str(bit) for bit in solution)
    return binary_string, model.objVal
  else:
    return None, None

#### Choose the solvers for experiments

In [ ]:
#@title Choose the solvers for experiments
brute_force_solver = False#@param {type:"boolean"}
minimum_eigen_solver = False#@param {type:"boolean"}
gurobi_solver = False#@param {type:"boolean"}
qiskit_qaoa_solver = False#@param {type:"boolean"}
new_nisq_solver = False#@param {type:"boolean"}
minimal_encoding_solver = True#@param {type:"boolean"}
dwave_annealer_solver = False#@param {type:"boolean"}


solver_flags = ''.join(map(str,map(int,[brute_force_solver, minimum_eigen_solver, gurobi_solver, qiskit_qaoa_solver, new_nisq_solver, minimal_encoding_solver, dwave_annealer_solver])))
solver_flags

#### Experimental Settings

In [ ]:
table_contents = []

QAOA_reps = 1

max_layers = 1000

# heights = np.arange(2, 17).tolist()
heights = [16,32]

scipy_optimizer_methods = ['Nelder-Mead', 'Powell', 'CG', 'BFGS', 'L-BFGS-B', 'TNC', 'COBYLA', 'SLSQP', 'trust-constr']
scipy_optimizer_methods = ['COBYLA']
other_optimizer_methods = ['Genetic']

seed = 111

# base_path = '/content/drive/MyDrive/Saarland/QAI/newnisq_seg/'
base_path = './'
report_filename = base_path + 'minimalEncoding_seg_report_' + solver_flags + '_' +  str(seed) + '.txt'

problem_instances = {}
ultimate_valid_probabilities = []
penultimate_valid_probabilities = []

# np.random.seed(seed=seed)

#### Executions

In [ ]:
for height in heights:
  # for width in widths:
  width = height
  print(f'height: {height}, width: {width}, n: {height*width}')
  # G, image = generate_binary_problem_instance(height, width)
  np.random.seed(seed=seed)
  G, image = generate_problem_instance(height, width)
  # print("Image Generated: ",image)
  start_time = time.time()
  if brute_force_solver:
    brute_force_solution, brute_force_value = bf_solver(G)
    print("BRUTE FORCE DONE!")
  else:
    brute_force_solution, brute_force_value = None, None
  brute_force_tte = (time.time() - start_time)

  start_time = time.time()
  if minimum_eigen_solver:
    minimum_eigen_solution, minimum_eigen_value = me2_solver(G)
    print("Minimum Eigensolver DONE!")
    # minimum_eigen_quality = get_quality(minimum_eigen_solution, brute_force_solution)
  else:
    minimum_eigen_solution, minimum_eigen_value = None, None
    # minimum_eigen_quality = None
  minimum_eigen_tte = (time.time() - start_time)

  start_time = time.time()
  if gurobi_solver:
    gurobi_solution, gurobi_value = gurobi_mincut_solver(G)
    print("Gurobi DONE!")
    # gurobi_quality = get_quality(gurobi_solution, brute_force_solution)
  else:
    gurobi_solution, gurobi_value = None, None
    # gurobi_quality = None
  gurobi_tte = (time.time() - start_time)

  start_time = time.time()
  if qiskit_qaoa_solver:
    qiskit_qaoa_solution, qiskit_qaoa_value = qaoa_solver(G, QAOA_reps)
    print("QAOA DONE!")
    # qiskit_qaoa_quality = get_quality(qiskit_qaoa_solution, brute_force_solution)
  else:
    qiskit_qaoa_solution, qiskit_qaoa_value = None, None
    # qiskit_qaoa_quality = None
  qiskit_qaoa_tte = (time.time() - start_time)
  start_time = time.time()
  if dwave_annealer_solver:
    dwave_annealer_solution, dwave_annealer_value = annealer_solver(G)
    print("ANNEALER DONE!")
    # dwave_annealer_quality = get_quality(dwave_annealer_solution, brute_force_solution)
  else:
    dwave_annealer_solution, dwave_annealer_value = None, None
    # dwave_annealer_quality = None
  dwave_annealer_tte = (time.time() - start_time)

  # start_time = time.time()
  # if new_nisq_genetic_solver:
    # new_nisq_ga_solution, new_nisq_ga_expectation_value, new_nisq_ga_cut_value = new_nisq_ga_solver2(G, population_size = min(10,int(2*G.number_of_nodes())),
                                                                                                      #  crossover_probability = 0.5,
                                                                                                      #  mutation_probability = 0.5,
                                                                                                      #  number_of_generations = min(10,int(2*G.number_of_nodes())))
    # print("NEW NISQ GENETIC ALGORITHM DONE!")
  # else:
    # new_nisq_ga_solution, new_nisq_ga_expectation_value, new_nisq_ga_cut_value = None, None, None
  # new_nisq_ga_tte = (time.time() - start_time)

  # initial_params = np.random.uniform(low=0.5, high=2*np.pi , size=(2**int(np.ceil(np.log2(int(G.number_of_nodes()))))))

  # if G.number_of_nodes() < max_layers:
    # max_layers = G.number_of_nodes()
  # else:
    # max_layers = 1000
  for n_layers in range(400, max_layers+1,100):
    nc = len(G.nodes())
    nr = ceil(log2(nc))
    nq = nr + 1
    np.random.seed(seed=seed)
    initial_params = np.random.uniform(low=-np.pi, high=np.pi, size=nq*n_layers)
    for scipy_optimizer_method in scipy_optimizer_methods:
      start_time = time.time()
      if minimal_encoding_solver:
        print(f"Executing QC with {n_layers} layers and {scipy_optimizer_method} optimizer for {height}*{height} image.")
        minimal_encoding_solution, minimal_encoding_value, minimal_encoding_cut_value, optimal_params = minimal_encoding(G, 
                                                                                                                        initial_params, 
                                                                                                                        n_layers, 
                                                                                                                        optimizer_method = scipy_optimizer_method)
        # minimal_encoding_solution, minimal_encoding_value, minimal_encoding_cut_value = new_nisq_algo_solver(G, scipy_optimizer_method, initial_params)
        print("New NISQ done for",scipy_optimizer_method,"optimizer")
        # new_nisq_quality = get_quality(new_nisq_cut_value, brute_force_value)
      else:
        new_nisq_solution, new_nisq_expectation_value, new_nisq_cut_value = None, None, None
        # new_nisq_quality = None
      minimal_encoding_tte = (time.time() - start_time)
      print(f"Appending the results of {height}*{height} image using QC with {n_layers} layers and {scipy_optimizer_method} optimizer.")
      row = []
      row.append(int(G.number_of_nodes()))
      row.append(int(height))
      row.append(int(width))
      if brute_force_solver:
        row.append(''.join(map(str, map(int, (brute_force_solution)))))
        row.append(brute_force_value)
        row.append(np.round(brute_force_tte,6))
      if minimum_eigen_solver:
        row.append(''.join(map(str, map(int, (minimum_eigen_solution)))))
        row.append(minimum_eigen_value)
        row.append(np.round(minimum_eigen_tte,6))
        # row.append(minimum_eigen_quality)
      if gurobi_solver:
        row.append(''.join(map(str, map(int, (gurobi_solution)))))
        row.append(gurobi_value)
        row.append(np.round(gurobi_tte,6))
        # row.append(minimum_eigen_quality)
      if qiskit_qaoa_solver:
        row.append(''.join(map(str, map(int, (qiskit_qaoa_solution)))))
        row.append(qiskit_qaoa_value)
        row.append(np.round(qiskit_qaoa_tte,6))
        # row.append(qiskit_qaoa_quality)
        row.append(QAOA_reps)
      if dwave_annealer_solver:
        row.append(''.join(map(str, map(int, (dwave_annealer_solution)))))
        row.append(dwave_annealer_value)
        row.append(np.round(dwave_annealer_tte,6))
        # row.append(dwave_annealer_quality)
      # if new_nisq_genetic_solver:
        # row.append(''.join(map(str, map(int, (new_nisq_ga_solution)))))
        # row.append(new_nisq_ga_cut_value)
        # row.append(np.round(new_nisq_ga_tte,6))
        # # row.append(new_nisq_ga_quality)
        # row.append(new_nisq_ga_expectation_value)
      if new_nisq_solver:
        row.append(''.join(map(str, map(int, (new_nisq_solution)))))
        row.append(new_nisq_cut_value)
        row.append(np.round(new_nisq_tte,6))
        # row.append(new_nisq_quality)
        row.append(new_nisq_expectation_value)
        row.append(scipy_optimizer_method)
      if minimal_encoding_solver:
        row.append(''.join(map(str, map(int, (minimal_encoding_solution)))))
        row.append(minimal_encoding_cut_value)
        row.append(np.round(minimal_encoding_tte,6))
        row.append(n_layers)
        row.append(minimal_encoding_value)
        row.append(scipy_optimizer_method)
        # row.append(''.join(map(str, map(float, (optimal_params)))))
      report_file_obj = open(os.path.join(report_filename),'a+')
      report_file_obj.write('__'.join(map(str,row))+'\n')
      report_file_obj.close()
  print('\n')

#### Display report

In [ ]:
def decode_binary_string(x, height, width):
  mask = np.zeros([height, width])
  for index,segment in enumerate(x):
    mask[index//width,index%width]=segment
  return mask

In [ ]:
base_path = '/content/drive/MyDrive/Saarland/QAI/newnisq_seg/'
base_path = './'


report_filename = base_path + 'minimalEncoding_seg_report_' + solver_flags + '_' +  str(seed) + '.txt'
report_filename

In [ ]:
brute_force_solver = bool(int(report_filename.split('_')[-2][0]))
minimum_eigen_solver = bool(int(report_filename.split('_')[-2][1]))
gurobi_solver = bool(int(report_filename.split('_')[-2][2]))
qiskit_qaoa_solver = bool(int(report_filename.split('_')[-2][3]))
dwave_annealer_solver = bool(int(report_filename.split('_')[-2][4]))
new_nisq_solver = bool(int(report_filename.split('_')[-2][5]))

report_file_obj = open(os.path.join(report_filename),'r')
table_contents = [line.replace('\n','').split('__') for line in report_file_obj.readlines()]

In [ ]:
print(brute_force_solver)
print(minimum_eigen_solver)
print(gurobi_solver)
print(qiskit_qaoa_solver)
print(dwave_annealer_solver)
print(new_nisq_solver)

In [ ]:
bf_solver_name = 'Brute Force (Exact)'
me_solver_name = 'Minimum Eigen (Classical)'
gurobi_solver_name = 'Gurobi (Classical SotA)'
qaoa_solver_name = 'Qiskit QAOA'
annealer_solver_name = 'D-Wave 2000Q'
new_nisq_solver_name = 'New NISQ Algorithm'

base_cols = ['No. of Pixels', 'Height', 'Width']
sub_cols = ['','','']

if brute_force_solver:
  base_cols = base_cols+[bf_solver_name]*3
  sub_cols=sub_cols+['Result', 'Value', 'TTE']
if minimum_eigen_solver:
  base_cols = base_cols+[me_solver_name]*3
  sub_cols=sub_cols+['Result', 'Value', 'TTE']
if gurobi_solver:
  base_cols = base_cols+[gurobi_solver_name]*3
  sub_cols=sub_cols+['Result', 'Value', 'TTE']
if qiskit_qaoa_solver:
  base_cols = base_cols+[qaoa_solver_name]*4
  sub_cols=sub_cols+['Result', 'Value', 'TTE', 'QAOA layers (p)']
if dwave_annealer_solver:
  base_cols = base_cols+[annealer_solver_name]*3
  sub_cols=sub_cols+['Result', 'Value', 'TTE']
if new_nisq_solver:
  base_cols = base_cols+[new_nisq_solver_name]*6
  sub_cols=sub_cols+['Result', 'Value', 'TTE', 'Layers', 'Expectation Value', 'Optimizer Method']

column_arrays = [base_cols, sub_cols]


#df = pd.DataFrame(table_contents, columns=table_headers)
df = pd.DataFrame(table_contents, columns=pd.MultiIndex.from_arrays(column_arrays))

In [ ]:
for row in range(len(df)):
    print(df[('New NISQ Algorithm','Result')][row])

In [ ]:
def truncate_long_string(s, max_length=20):
    return s if len(s) <= max_length else s[:max_length] + "..."

# Apply this function to the specific column(s)
for col in df.columns:
  if len(col) == 2 and col[1] == 'Result':
    df[col] = df[col].apply(lambda x: truncate_long_string(x))

In [ ]:
# def highlight_row(s):
#     should_highlight = abs(round(float(s[(new_nisq_solver_name, 'Value')]), 3)) == abs(round(float(s[(gurobi_solver_name, 'Value')]), 3))
#     return ['background-color: #606f60' if should_highlight else '' for _ in s]

# df = df.apply(highlight_row, axis=1)
# df

In [ ]:
def style_df(df):
    # Styles for hover and headers
    cell_hover = {'selector': 'tr:hover', 'props': [('background-color', '#707070')]}  # Dark pale color for hover
    headers = {'selector': 'th', 'props': 'background-color: #1D1D1D; color: white;'}

    # Column separator style
    column_separator = {'selector': 'td, th', 'props': 'border-right: 1px solid white;'}

    # Apply the styles
    styled_df = df.style.set_table_styles([cell_hover, headers, column_separator])

    # Style to hide the border for the last column to prevent a double line at the end of the table
    hide_last_border = {'selector': 'td:last-child, th:last-child', 'props': 'border-right: none;'}

    # Combine all styles
    styles = [cell_hover, headers, column_separator, hide_last_border]
    styled_df = styled_df.set_table_styles(styles, overwrite=False)

    return styled_df

styled_df = style_df(df)#.apply(highlight_row, axis=1)

styled_df